<a href="https://colab.research.google.com/github/nsmq-ai/nsmqai/blob/kojomensahonums-add-stt-inference-notebook/STT_Inference_API_Server_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import and install the required libraries

%%capture
!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install tabulate
!pip install pydub
import torch
import numpy as np
import whisper
import jiwer
import time
import pandas as pd
from tabulate import tabulate
from pydub import AudioSegment
import os

In [2]:
# Load whisper model

%%capture
whisper.load_model("medium.en")


In [3]:
torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = whisper.load_model("medium.en", device = DEVICE) # Select whisper model size (tiny, base, small, medium, large)
# print(
#       f"Model is {'multilingual ' if model.is_multilingual else 'English only'}"
#       f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."

def transcribe(path_to_audio):

  # Load audio
  audio = whisper.load_audio(path_to_audio)

  # Measure start time
  #start = time.time()

  # Transcribe audio
  result = model.transcribe(audio)

  # Measure duration of transcription time
  #transcription_time = time.time()-start
  #print(f"The transcription time is {transcription_time} seconds")

  # Print transcript
  return result["text"]

In [4]:
# Install required libraries for web api
!pip -q install fastapi
!pip -q install pyngrok
!pip -q install uvicorn
!pip -q install nest_asyncio
!pip -q install python-multipart

In [5]:
# Import libraries
import uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
from pydantic import BaseModel
import nest_asyncio

import shutil

nest_asyncio.apply()

In [6]:
# Import models for serialisation/ deserialisation
from pydantic import BaseModel
import base64
import io
import wave


class AudioBytes(BaseModel):
  data: bytes
  filename: str


app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get("/get-transcript")
async def get_transcript(audio: AudioBytes):
  decoded_data = base64.b64decode(audio.data)

  # Write bytes data to a .wav file
  with io.BytesIO(decoded_data) as audio_file:
    with wave.open(audio_file, "wb") as wav:
      wav.setnchannels(1)
      wav.setsampwidth(2)
      wav.setframerate(16000)

      # Write .wav files
      wav.writeframes(decoded_data)

  # Save the audio file with the custom name
    audio_filename = audio.filename
    with open(audio_filename, "wb") as file:
        file.write(decoded_data)

   # Get transcript and delete temporary audio file
  print("audio_filename ",audio_filename)
  transcript = transcribe(audio_filename)
  os.remove(audio_filename)

  return {"transcript": transcript}

@app.get("/")
async def home():
  return {"msg":"Hello from STT."}



In [7]:
# Attach personal token
!ngrok config add-authtoken <place_your_ngrok_auth_token_here>

In [8]:
# Link to model in API ??
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)
uvicorn.run(app, port=8000)